In [143]:
import requests,pandas,geopandas

# There are 80 sirens in Maui County, requesting higher OBJID numbers don't return anything
OBJID_RANGE = range(0,100)
objid_str_comma_sep = ','.join([str(x) for x in OBJID_RANGE])
objid_str_comma_sep

GPS_COORDS = "epsg:4326"

params = {
    'f': 'json',
    'objectIds': objid_str_comma_sep,
    'outFields': 'Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Siren,Solar,OBJECTID,GlobalID',
    'outSR': '102100',
    'returnM': 'true',
    'returnZ': 'true',
    'spatialRel': 'esriSpatialRelIntersects',
    'where': '1=1',
}

response = requests.get(
    'https://services3.arcgis.com/fsrDo0QMPlK9CkZD/arcgis/rest/services/Tsunami_Sirens/FeatureServer/0/query',
    params=params
)

maui_sirens = pandas.DataFrame(pandas.DataFrame(response.json()["features"])["attributes"].to_list())
print(f"{maui_sirens.shape[0]} sirens in the dataset")
print(str(maui_sirens[maui_sirens["Lat"].notna()].shape[0])+" sirens with coordinate info")
print(str(maui_sirens[maui_sirens["Range_ft"].notna()].shape[0])+" sirens with range info")
print("Sirens missing coordinate information:",maui_sirens[maui_sirens["Lat"].isna()]["Name"].to_list())
print("(Manual review confirmed none of these are in the Lahaina burn perimeter)")

maui_sirens["dec_num"] = pandas.to_numeric(maui_sirens["Decibel"],errors="coerce")
maui_sirens["range_num"] = maui_sirens["Range_ft"].apply(lambda x: str(x).split(" ")[0])
maui_sirens["range_num"] = pandas.to_numeric(maui_sirens["range_num"], errors="coerce")
maui_sirens_geo = geopandas.GeoDataFrame(maui_sirens, geometry=geopandas.points_from_xy(maui_sirens.Lon, maui_sirens.Lat))

# Set the CRS
maui_sirens_geo.crs = GPS_COORDS
maui_sirens_geo.to_file("data/processed/maui_sirens--coord_map.shp")

maui_sirens_buff = maui_sirens_geo[maui_sirens_geo["range_num"] > 0]

# Change to a foot-based CRS to run the buffer 
maui_sirens_buff.to_crs("esri:102662", inplace=True)
maui_sirens_buff['geometry'] = maui_sirens_buff.apply(lambda x: x.geometry.buffer(x.range_num), axis=1)

# Change CRS back
maui_sirens_buff.to_crs(GPS_COORDS, inplace=True)
maui_sirens_buff.to_file("data/processed/maui_sirens--buffer.shp")


80 sirens in the dataset
71 sirens with coordinate info
69 sirens with range info
Sirens missing coordinate information: ['Paia Youth & Culture Club', 'Makena Waste Water Treatment', 'Big Beach', 'Kihei Community Center', 'Hamoa II', 'Napili', 'Mehana Estates', 'Papohaku Beach Park', 'One Alii Park']
(Manual review confirmed none of these are in the Lahaina burn perimeter


/opt/homebrew/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [2]:
import pandas

sirens = pandas.read_csv("data/source/lahaina/department-of-defense-state-civil-defense-emergency-siren-locations-csv.csv")
sirens

,COUNTY,UNIQUE_ID,LOCATION,DECIBEL,Location 1
0,MAUI,MA304,Hamoa,115,"(20.719, -155.988)"
1,HAWAII,HA917,Ahulanui Beach Park,119,"(19.468, -154.832)"
2,MAUI,MA121,Wailea #1-Mokapu-Ulua BP,124,"(20.691, -156.443)"
3,KAUAI,KA505,Poipu Bch Pk,124,"(21.874, -159.454)"
4,HONOLULU,OA424,Hauula Beach Park,118,"(21.613, -157.911)"
...,...,...,...,...,...
335,HONOLULU,OA405,Lanikai/Aala Dr/Mokulua,118,"(21.388, -157.713)"
336,HONOLULU,OA120,McKinley HS,119,"(21.297, -157.85)"
337,HONOLULU,OA127,Honolulu Comm. College,115,"(21.32, -157.868)"
338,HONOLULU,OA201,Sunset Beach,121,"(21.674, -158.04)"


In [176]:
sirens[sirens["COUNTY"]=="MAUI"]

,COUNTY,UNIQUE_ID,LOCATION,DECIBEL,Location 1,latitude,longitude
0,MAUI,MA304,Hamoa,115,"(20.719, -155.988)",20.719,-155.988
2,MAUI,MA121,Wailea #1-Mokapu-Ulua BP,124,"(20.691, -156.443)",20.691,-156.443
9,MAUI,MA408,Kaanapali #1 STP,126,"(20.945, -156.687)",20.945,-156.687
15,MAUI,MA103,Paukukalo (IGE),115,"(20.903, -156.481)",20.903,-156.481
21,MAUI,MA125,Makena,115,"(20.661, -156.439)",20.661,-156.439
...,...,...,...,...,...,...,...
311,MAUI,MA119,"Pukalani, Kula",119,"(20.839, -156.341)",20.839,-156.341
318,MAUI,MA114,"Keawekapu, Kihei",115,"(20.7, -156.444)",20.7,-156.444
321,MAUI,MA401,Olowalu,121,"(20.811, -156.622)",20.811,-156.622
325,MAUI,MA503,"Kamiloloa, Mol",115,"(21.078, -156.993)",21.078,-156.993


In [8]:
import geopandas

sirens["latitude"] = sirens["Location 1"].apply(lambda x: x[1:-1].split(",")[0])
sirens["longitude"] = sirens["Location 1"].apply(lambda x: x[1:-1].split(",")[1])

sirens_geo = geopandas.GeoDataFrame(sirens, geometry=geopandas.points_from_xy(sirens.longitude, sirens.latitude))
sirens_geo.crs = "EPSG:4326"

In [12]:
sirens_geo.to_file("data/processed/lahaina/sirens.shp")

In [24]:
import requests

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
}

params = {
    'f': 'json',
    'cacheHint': 'true',
    'maxRecordCountFactor': '4',
    'resultOffset': '0',
    'resultRecordCount': '8000',
    'where': '1=1',
    'orderByFields': 'OBJECTID',
    'outFields': 'OBJECTID',
    'outSR': '102100',
    'spatialRel': 'esriSpatialRelIntersects',
}

response = requests.get(
    'https://services3.arcgis.com/fsrDo0QMPlK9CkZD/arcgis/rest/services/Tsunami_Sirens/FeatureServer/0/query',
    params=params,
    headers=headers,
)

In [49]:
import requests

objid_range = range(1,200)
objid_str_comma_sep = ','.join([str(x) for x in objid_range])
objid_str_comma_sep

headers = {
    'authority': 'services3.arcgis.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.6',
    'origin': 'https://www.mauisirens.com',
    'referer': 'https://www.mauisirens.com/',
    'sec-ch-ua': '"Not/A)Brand";v="99", "Brave";v="115", "Chromium";v="115"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
}

params = {
    'f': 'json',
    'objectIds': objid_str_comma_sep,
    'outFields': 'Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Siren,Solar,OBJECTID,GlobalID',
    'outSR': '102100',
    'returnM': 'true',
    'returnZ': 'true',
    'spatialRel': 'esriSpatialRelIntersects',
    'where': '1=1',
}

response_2 = requests.get(
    'https://services3.arcgis.com/fsrDo0QMPlK9CkZD/arcgis/rest/services/Tsunami_Sirens/FeatureServer/0/query',
    params=params,
    headers=headers,
)

In [56]:
maui_sirens_1 = sirens[sirens["COUNTY"]=="MAUI"]
maui_sirens_1

,COUNTY,UNIQUE_ID,LOCATION,DECIBEL,Location 1,latitude,longitude
0,MAUI,MA304,Hamoa,115,"(20.719, -155.988)",20.719,-155.988
2,MAUI,MA121,Wailea #1-Mokapu-Ulua BP,124,"(20.691, -156.443)",20.691,-156.443
9,MAUI,MA408,Kaanapali #1 STP,126,"(20.945, -156.687)",20.945,-156.687
15,MAUI,MA103,Paukukalo (IGE),115,"(20.903, -156.481)",20.903,-156.481
21,MAUI,MA125,Makena,115,"(20.661, -156.439)",20.661,-156.439
...,...,...,...,...,...,...,...
311,MAUI,MA119,"Pukalani, Kula",119,"(20.839, -156.341)",20.839,-156.341
318,MAUI,MA114,"Keawekapu, Kihei",115,"(20.7, -156.444)",20.7,-156.444
321,MAUI,MA401,Olowalu,121,"(20.811, -156.622)",20.811,-156.622
325,MAUI,MA503,"Kamiloloa, Mol",115,"(21.078, -156.993)",21.078,-156.993


In [114]:
sirens_2

,Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Siren,Solar,OBJECTID,GlobalID
0,115,Maui,20.909302,-156.490795,MOD3012,Waiehu,1.700439e+06,209224.611128,2200 Feet,MA101,YES,2,20b4796e-d66b-4793-b372-d0661817b5b4
1,115,Maui,20.915077,-156.494045,MOD3012,Waiehu Beach Road,1.699308e+06,211300.623052,2200 Feet,MA102,YES,3,fc1083fd-4963-4563-b077-5cc18d6ce72d
2,115,Maui,20.903390,-156.481863,EOWS1212,Nisei Veterans Memorial Center,1.702995e+06,206625.499856,2200 Feet,MA103,NO,4,b1ec6138-8260-4f06-9c23-d67eccf88b59
3,121,Maui,20.891741,-156.462600,MOD6024,Kahului Sewer Baseyard,1.710068e+06,202860.634354,3400 Feet,MA104,YES,6,9fc0ae8c-77ef-418c-9300-c1f6d2b52bab
4,124,Maui,20.890905,-156.445406,EOWS612,NASKA Airport Road,1.715945e+06,202567.629691,4800 Feet,MA105,NO,7,59c88e14-0a38-4889-aa30-6ce38e5c1ae4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,121,Lanai,20.746880,-156.896500,DSA6(2),Manele Bay,1.561903e+06,150242.879682,3400 Feet,MA202,YES,80,51c1464e-73f5-4b26-8245-f568c1989275
76,None,Maui,20.820097,-156.454853,None,Puunene,NaN,NaN,None,MA148,Yes,81,89f01b28-d9f2-4146-8f10-da96191f850a
77,None,Maui,20.834226,-156.329243,None,Kualono,NaN,NaN,None,MA155,Yes,82,ba35c755-1131-4fdd-973e-8d5c7a66b018
78,115,Maui,20.900082,-156.491234,MOD3012,Papohaku Park,1.700187e+06,205817.839726,2200 Feet,MA133,YES,84,a3c9f426-5e4b-4828-96d6-5771c7a8a054


In [71]:
sirens_2 = pandas.DataFrame(pandas.DataFrame(response_2.json()["features"])["attributes"].to_list())

sirens_2.to_csv("data/processed/lahaina/maui_sirens_scrape.csv")
geo_sirens = geopandas.GeoDataFrame(sirens_2, geometry=geopandas.points_from_xy(sirens_2.Lon, sirens_2.Lat))
geo_sirens.to_file("data/processed/lahaina/maui_sirens_scrape.shp")

In [111]:
geo_sirens["range_num"].describe()

count      68.000000
mean     2837.867647
std       974.550151
min      2200.000000
25%      2200.000000
50%      2200.000000
75%      3400.000000
max      5600.000000
Name: range_num, dtype: float64

In [112]:
geo_sirens["dec_num"] = pandas.to_numeric(geo_sirens["Decibel"],errors="coerce")
geo_sirens["dec_num"].describe()

count     69.000000
mean     117.565217
std        3.867201
min      115.000000
25%      115.000000
50%      115.000000
75%      121.000000
max      128.000000
Name: dec_num, dtype: float64

In [98]:
geo_sirens["dec_num"] = pandas.to_numeric(geo_sirens["Decibel"],errors="coerce")
geo_sirens["dec_num"].describe()
geo_sirens["range_num"] = geo_sirens["Range_ft"].apply(lambda x: str(x).split(" ")[0])
geo_sirens["range_num"] = pandas.to_numeric(geo_sirens["range_num"], errors="coerce")
geo_sirens_buff = geo_sirens[geo_sirens["range_num"] > 0]
#set crs to ESRI:102662
geo_sirens_buff.crs = "epsg:4326"
geo_sirens_buff.to_crs("esri:102662", inplace=True)
geo_sirens_buff['geometry'] = geo_sirens_buff.apply(lambda x: x.geometry.buffer(x.range_num), axis=1)
geo_sirens_buff.to_crs("epsg:4326", inplace=True)
geo_sirens_buff.to_file("data/processed/lahaina/buffer.shp")

/opt/homebrew/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [175]:
geo_sirens_buff

,Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Siren,Solar,OBJECTID,GlobalID,geometry,range_num
0,115,Maui,20.909302,-156.490795,MOD3012,Waiehu,1.700439e+06,209224.611128,2200 Feet,MA101,YES,2,20b4796e-d66b-4793-b372-d0661817b5b4,"POLYGON ((-156.48435 20.90930, -156.48438 20.9...",2200.0
1,115,Maui,20.915077,-156.494045,MOD3012,Waiehu Beach Road,1.699308e+06,211300.623052,2200 Feet,MA102,YES,3,fc1083fd-4963-4563-b077-5cc18d6ce72d,"POLYGON ((-156.48760 20.91507, -156.48763 20.9...",2200.0
2,115,Maui,20.903390,-156.481863,EOWS1212,Nisei Veterans Memorial Center,1.702995e+06,206625.499856,2200 Feet,MA103,NO,4,b1ec6138-8260-4f06-9c23-d67eccf88b59,"POLYGON ((-156.47542 20.90338, -156.47545 20.9...",2200.0
3,121,Maui,20.891741,-156.462600,MOD6024,Kahului Sewer Baseyard,1.710068e+06,202860.634354,3400 Feet,MA104,YES,6,9fc0ae8c-77ef-418c-9300-c1f6d2b52bab,"POLYGON ((-156.45264 20.89173, -156.45269 20.8...",3400.0
4,124,Maui,20.890905,-156.445406,EOWS612,NASKA Airport Road,1.715945e+06,202567.629691,4800 Feet,MA105,NO,7,59c88e14-0a38-4889-aa30-6ce38e5c1ae4,"POLYGON ((-156.43134 20.89089, -156.43141 20.8...",4800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,115,Molokai,21.190102,-157.243439,MOD3012,Kaluakoi,1.443929e+06,311550.330360,2200 Feet,MA510,YES,75,d3a57099-ad04-4b6d-9335-63f2d02a64ae,"POLYGON ((-157.23698 21.19012, -157.23701 21.1...",2200.0
73,126,Molokai,21.190562,-156.980001,100AT,Kalaupapa,1.533393e+06,313287.682857,2575 Feet,MA601,,78,199a45f1-5256-41da-9d30-809db305acfc,"POLYGON ((-156.97244 21.19058, -156.97248 21.1...",2575.0
74,115,Lanai,20.822429,-156.917129,MOD3012,Lanai City,1.554887e+06,177708.148898,2200 Feet,MA201,YES,79,02051c81-5b90-4389-9e9d-27ca6ffea04f,"POLYGON ((-156.91069 20.82244, -156.91072 20.8...",2200.0
75,121,Lanai,20.746880,-156.896500,DSA6(2),Manele Bay,1.561903e+06,150242.879682,3400 Feet,MA202,YES,80,51c1464e-73f5-4b26-8245-f568c1989275,"POLYGON ((-156.88655 20.74689, -156.88660 20.7...",3400.0


0     2200.0
32    2200.0
40    2200.0
41    2200.0
43    2200.0
       ...  
71       NaN
72       NaN
76       NaN
77       NaN
79       NaN
Name: range_num, Length: 80, dtype: float64

In [62]:
compare = maui_sirens_1.join(sirens_2.set_index("Siren"), on ="UNIQUE_ID", how="outer")
compare[compare['Island'].isnull()]

,COUNTY,UNIQUE_ID,LOCATION,DECIBEL,Location 1,latitude,longitude,Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Solar,OBJECTID,GlobalID
251.0,MAUI,MA138,Mai Poina Oe lau,121.0,"(20.771, -156.459)",20.771,-156.459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274.0,MAUI,MA409,Kaanapali #2,126.0,"(20.917, -156.689)",20.917,-156.689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
compare[compare['COUNTY'].isnull()]

,COUNTY,UNIQUE_ID,LOCATION,DECIBEL,Location 1,latitude,longitude,Decibel,Island,Lat,Lon,Model,Name,POINT_X,POINT_Y,Range_ft,Solar,OBJECTID,GlobalID
NaN,NaN,MA137,NaN,NaN,NaN,NaN,NaN,121,Maui,20.774354,-156.460316,DSA6/6/4,Mai Poina Oe Lau Park,1.710908e+06,160227.451904,3400 Feet,YES,39.0,82378239-9ded-4a73-a41a-b23481e2f8de
NaN,NaN,MA142,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Paia Youth & Culture Club,1.736866e+06,211372.943445,None,None,40.0,1091e0a4-10aa-4644-abcf-c2dfd610bdae
NaN,NaN,MA156,NaN,NaN,NaN,NaN,NaN,,Maui,0.000000,0.000000,,Wailea-Makena,1.718487e+06,122838.169728,3400 Feet,,41.0,3e366c3b-beb0-4ea1-af6f-e092105bde96
NaN,NaN,MA157,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Makena Waste Water Treatment,1.720970e+06,114045.678126,None,None,42.0,14590cd0-c5c7-4443-aaa5-8fae04af651a
NaN,NaN,MA158,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Big Beach,1.716420e+06,109751.524313,None,None,43.0,5f1b809c-dc36-4b77-b69f-dd54a7d551b3
NaN,NaN,MA159,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Kihei Community Center,1.715681e+06,150962.556392,None,None,44.0,47f8572e-161e-4826-8ba8-547b65d35afb
NaN,NaN,MA304A,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Hamoa II,1.873479e+06,142213.872774,None,None,49.0,c03fccfa-f735-48d0-957a-f408f6f53712
NaN,NaN,MA406A,NaN,NaN,NaN,NaN,NaN,115,Maui,NaN,NaN,None,Napili,1.640381e+06,239389.242071,2200 Feet,None,58.0,a9d3392f-e958-4886-9f8b-24e2035ea68c
NaN,NaN,MA414,NaN,NaN,NaN,NaN,NaN,None,Maui,NaN,NaN,None,Mehana Estates,1.649005e+06,239751.808931,None,None,65.0,bc3170ea-4655-4ebc-b3fd-c1c7da953d70
NaN,NaN,MA513,NaN,NaN,NaN,NaN,NaN,None,Molokai,NaN,NaN,None,Papohaku Beach Park,1.441429e+06,305374.959509,None,None,76.0,db379921-0913-4fed-ad72-c3fea8e87360


In [103]:
response.json()

{'objectIdFieldName': 'OBJECTID',
 'uniqueIdField': {'name': 'OBJECTID', 'isSystemMaintained': True},
 'globalIdFieldName': 'GlobalID',
 'serverGens': {'minServerGen': 2533825, 'serverGen': 2533825},
 'geometryType': 'esriGeometryPoint',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'fields': [{'name': 'OBJECTID',
   'type': 'esriFieldTypeOID',
   'alias': 'OBJECTID',
   'sqlType': 'sqlTypeOther',
   'domain': None,
   'defaultValue': None}],
 'features': [{'attributes': {'OBJECTID': 2},
   'geometry': {'x': -17420478.422869626, 'y': 2381064.4574811263}},
  {'attributes': {'OBJECTID': 3},
   'geometry': {'x': -17420846.601687443, 'y': 2381745.962876673}},
  {'attributes': {'OBJECTID': 4},
   'geometry': {'x': -17419645.60316478, 'y': 2380210.8333323975}},
  {'attributes': {'OBJECTID': 6},
   'geometry': {'x': -17417340.292570744, 'y': 2378972.9742292534}},
  {'attributes': {'OBJECTID': 7},
   'geometry': {'x': -17415423.853108436, 'y': 2378874.30630851}},
  {'attributes':

In [32]:
temp = pandas.DataFrame(response.json()["features"])
json_coords = pandas.DataFrame(temp["geometry"].to_list())
geo_json_coords = geopandas.GeoDataFrame(temp, geometry=geopandas.points_from_xy(json_coords["x"], json_coords["y"]))

In [34]:
geo_json_coords.to_file("data/processed/scraped-sirens.shp")

In [141]:
hi_blocks = geopandas.read_file("data/source/lahaina/tl_rd22_15_tabblock20/tl_rd22_15_tabblock20.shp")
maui_blocks = hi_blocks[hi_blocks["COUNTYFP20"] == "009"]
maui_blocks.to_file("data/processed/maui_blocks.shp")

In [174]:
RENAME_CENSUS = {" !!Total:":"total_groupqt_pop",' !!Total:!!Institutionalized population:':"total_inst_pop",' !!Total:!!Institutionalized population:!!Correctional facilities for adults':"adult_corr",' !!Total:!!Institutionalized population:!!Juvenile facilities':"juv_corr",' !!Total:!!Institutionalized population:!!Nursing facilities/Skilled-nursing facilities':"snf",' !!Total:!!Institutionalized population:!!Other institutional facilities':"other_inst",' !!Total:!!Noninstitutionalized population:':"total_noninst_pop",' !!Total:!!Noninstitutionalized population:!!College dormitories (includes college quarters off campus)':"college_dorms",' !!Total:!!Noninstitutionalized population:!!Military quarters':"military_quarters",' !!Total:!!Noninstitutionalized population:!!Other noninstitutional facilities':"other_noninst"}
groupqt = pandas.read_csv("data/source/lahaina/DECENNIALPL2020.P5_2023-08-15T094425/DECENNIALPL2020.P5-Data.csv")
groupqt.columns = groupqt.iloc[0]
groupqt = groupqt[1:]
qt_cols = pandas.DataFrame([groupqt.columns]).transpose()
qt_cols = qt_cols[(~qt_cols[0].str.contains("Annotation",na=False))&qt_cols[0].notna()]
qt_cols = qt_cols[0].tolist()
num_cols = qt_cols[2:]
groupqt[num_cols] = groupqt[num_cols].apply(pandas.to_numeric, errors='coerce')
groupqt['join_key'] = groupqt["Geography"].apply(lambda x: x.split("US")[1])
joined = maui_blocks.join(groupqt.set_index('join_key')[qt_cols], on='GEOID20')
joined = joined.rename(columns=RENAME_CENSUS)
joined.to_file("data/processed/lahaina/maui--blocks--groupqt.shp")

/var/folders/n1/qf74t1dj555cgs6f_mhz9nm0m6xrwl/T/ipykernel_54321/1132269462.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined.to_file("data/processed/lahaina/maui--blocks--groupqt.shp")
